<h1><strong>Import Library</strong></h1>

In [1]:
import numpy as np
import pandas as pd
import torch
import os
import shutil
import itertools
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset, SubsetRandomSampler
from torchvision.datasets import DatasetFolder, VisionDataset
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold

from tqdm.auto import tqdm
import random

In [2]:
myseed = 123456 #just a random number
_exp_name = "sample"
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

<h1><strong>Create Augmentation for the Dataset </strong></h1>

In [3]:
validation_tfm = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor()
    ])

train_tfm = transforms.Compose([
    
    transforms.Resize((128,128)),
    
    transforms.ColorJitter(brightness=0.15, saturation=0.15, contrast=0.15),
    transforms.RandomAffine(degrees=20, translate=(0.15,0.15), scale=(1,1.2)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()    
])

<h1><strong>Dataset Class</strong></h1>

In [21]:
class PinappleDataset(Dataset):
    def __init__(self,path,tfm,files = None):
        super(PinappleDataset).__init__()
        self.path = path #path for the dataset 
        self.files = path
        if files!= None:
            self.files = files
        print(f"One {path} sample", self.files[0])
        self.transform = tfm #transform or the image augmentation
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        fname = self.files[idx]
        im = Image.open(fname)
        im = self.transform(im)
        
        label = fname.split("/")[-2]
        
        if label == "stage_501":
            return im, 0
        elif label == "stage_505":
            return im, 1
        elif label == "stage_509":
            return im, 2
        elif label == "stage_601":
            return im, 3
        elif label == "stage_605":
            return im, 4
        elif label == "stage_609":
            return im, 5
        elif label == "stage_701":
            return im, 6
        elif label == "stage_705":
            return im, 7
        elif label == "stage_709":
            return im, 8
        else:
            print(f"{fname}:No Label")

<h1><strong>CNN</strong></h1>

In [5]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # input 維度 [3, 128, 128]
#         (128x8192 and 16384x256)
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),  # [64, 128, 128]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [64, 64, 64]

            nn.Conv2d(64, 128, 3, 1, 1), # [128, 64, 64]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [128, 32, 32]

            nn.Conv2d(128, 256, 3, 1, 1), # [256, 32, 32]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [256, 16, 16]

            nn.Conv2d(256, 512, 3, 1, 1), # [512, 16, 16]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 8, 8]
            
            nn.Conv2d(512, 512, 3, 1, 1), # [512, 8, 8]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 4, 4]
        )
        self.fc = nn.Sequential(
            nn.Linear(128*8*8, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 9) #We have 9 classes 
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)

<h2>Function to move pictures to train and validation</h2>

In [16]:
def dataMover(filesName, destination):
    for i in filesName:
        name = str(i.split("/")[-1])
        classId = str(i.split("/")[-2])
        classId1 = str(classId.split("_"))[0]
        classId2 = str(classId.split("_"))[1]
        nameFile = classId1 + classId2 + "_" + name
        shutil.move(i, os.path.join(destination, nameFile))

<h2>Create DataLoader</h2>

In [25]:
batch_size = 128 
_dataset_dir = "Pineapple_image" #dataset path

train_path = "./Dataset/Train" #train directory
valid_path = "./Dataset/Validation" #valid directory

filePath = []

#split the dataset to train and validation
directPath = [x for x in os.listdir(_dataset_dir)] #need to change
for directory in directPath:
    for fileName in os.listdir(os.path.join(_dataset_dir,directory)):
        filePath.append(_dataset_dir + "/"+ directory + "/" + fileName)

train_dataset, validation_dataset = train_test_split(filePath, random_state = 104, test_size = 0.2, shuffle= True)

dataTrain = PinappleDataset(train_dataset, tfm= train_tfm)
dataValid = PinappleDataset(validation_dataset, tfm= validation_tfm)

dataset = ConcatDataset([dataTrain,dataValid])

One ['Pineapple_image/stage_605/DSC_0147.JPG', 'Pineapple_image/stage_709/IMG_4530.JPG', 'Pineapple_image/stage_601/IMG_0551.JPG', 'Pineapple_image/stage_501/IMG_8869.JPG', 'Pineapple_image/stage_505/IMG_20210112_120437.jpg', 'Pineapple_image/stage_501/IMG_3927.JPG', 'Pineapple_image/stage_501/DSC_0507.JPG', 'Pineapple_image/stage_709/IMG_9667.JPG', 'Pineapple_image/stage_509/IMG_0254.JPG', 'Pineapple_image/stage_505/IMG_4472.JPG', 'Pineapple_image/stage_501/IMG_3779.JPG', 'Pineapple_image/stage_705/IMG_9486.JPG', 'Pineapple_image/stage_609/IMG_2777.JPG', 'Pineapple_image/stage_601/IMG_3521.JPG', 'Pineapple_image/stage_609/IMG_5938.JPG', 'Pineapple_image/stage_705/IMG_1315.JPG', 'Pineapple_image/stage_509/IMG_3468.JPG', 'Pineapple_image/stage_509/IMG_0613.JPG', 'Pineapple_image/stage_601/IMG_5616.JPG', 'Pineapple_image/stage_501/IMG_7245.JPG', 'Pineapple_image/stage_705/IMG_0345.JPG', 'Pineapple_image/stage_501/IMG_2082.JPG', 'Pineapple_image/stage_509/IMG_0509.JPG', 'Pineapple_image/s

<h1><strong>Training Loop</strong></h1>

In [26]:
device = "cuda" if torch.cuda.is_available() else "cpu"

n_epochs = 100
patience = 10 #if no imporvement early stop

model = Classifier().to(device)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr = 0.0003, weight_decay= 1e-5)

stale = 0
best_acc = 0

k = 5
splits = KFold(n_splits= k, shuffle= True, random_state= 42)

for fold, (train_idx, val_idx) in enumerate(splits.split(np.arange(len(dataset)))):
    train_sampler = SubsetRandomSampler(train_idx)
    val_sampler = SubsetRandomSampler(val_idx)
    
    train_loader = DataLoader(dataset, batch_size = batch_size, sampler = train_sampler)
    val_loader = DataLoader(dataset, batch_size = batch_size, sampler = val_sampler)
    
    for epoch in range(n_epochs):
        model.train()
        
        train_loss = []
        train_acs = []
        
        for batch in tqdm(train_loader):
            
            imgs, labels = batch
            
            logits = model(imgs.to(device))
            
            loss = criterion(logits, labels.to(device))
            
            optimizer.zero_grad()
            loss.backward()
            
            grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm= 10)
            
            optimizer.step()
            
            acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
            
            train_loss.append(loss.item())
            train_acs.append(acc)
            
        train_loss = sum(train_loss) / len(train_loss)
        train_acc = sum(train_acs) / len(train_acs)
        
         # Print the information.
        print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

        # ---------- Validation ----------
        # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
        model.eval()

        # These are used to record information in validation.
        valid_loss = []
        valid_accs = []
        
        for batch in tqdm(val_loader):
            
            imgs, labels = batch
            
            with torch.no_grad():
                logits = model(imgs.to(device))

            loss = criterion(logits, labels.to(device))
    
            acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

            valid_loss.append(loss.item())
            valid_accs.append(acc)
            
        # The average loss and accuracy for entire validation set is the average of the recorded values.
        valid_loss = sum(valid_loss) / len(valid_loss)
        valid_acc = sum(valid_accs) / len(valid_accs)

        # Print the information.
        print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")
        
        # update logs
        if valid_acc > best_acc:
            with open(f"./{_exp_name}_log.txt","a"):
                print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best")
        else:
            with open(f"./{_exp_name}_log.txt","a"):
                print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

        # save models
        if valid_acc > best_acc:
            print(f"Best model found at epoch {epoch}, saving model")
            torch.save(model.state_dict(), f"{_exp_name}_best_.ckpt") # only save best to prevent output memory exceed error
            best_acc = valid_acc
            stale = 0
        else:
            stale += 1
            if stale > patience:
                print(f"No improvment {patience} consecutive epochs, early stopping")
                break

  0%|          | 0/12 [00:00<?, ?it/s]

[ Train | 001/100 ] loss = 1.66660, acc = 0.39388


  0%|          | 0/3 [00:00<?, ?it/s]

[ Valid | 001/100 ] loss = 2.30653, acc = 0.11478
[ Valid | 001/100 ] loss = 2.30653, acc = 0.11478 -> best
Best model found at epoch 0, saving model


  0%|          | 0/12 [00:00<?, ?it/s]

...

[ Valid | 011/100 ] loss = 0.28387, acc = 0.89443
[ Valid | 011/100 ] loss = 0.28387, acc = 0.89443


  0%|          | 0/12 [00:00<?, ?it/s]

[ Train | 012/100 ] loss = 0.07183, acc = 0.97005


  0%|          | 0/3 [00:00<?, ?it/s]

[ Valid | 012/100 ] loss = 0.19726, acc = 0.94491
[ Valid | 012/100 ] loss = 0.19726, acc = 0.94491
No improvment 10 consecutive epochs, early stopping


In [ ]:
# util functions
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
      cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        # print("Normalized confusion matrix")
    else:
      pass
        # print('Confusion matrix, without normalization')

    # print(cm)
    plt.figure(figsize=(6, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    # plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
# from sklearn.metrics import confusion_matrix

# classes = ['Stage 501', 'Stage 505', 'Stage 509', 'Stage 601', 'Stage 605', 'Stage 609', 'Stage 701', 'Stage 705', 'Stage 709']

# #y_test: Ground Truth
# #y_pred: Prediction
# matrix = confusion_matrix(y_test, y_pred) 
# plot_confusion_matrix(matrix, classes=classes, normalize=True)